In [2]:
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords

[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1129)>


In [4]:
from wordcloud import wordcloud
import matplotlib.pyplot as plt 
from IPython import display
import time

In [7]:
from pyspark.sql import SparkSession as ss 
from pyspark.sql import functions as f 

spark = ss.builder.master('local[*]').appName('wordcloud').getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/19 03:35:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/10/19 03:35:56 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [8]:
spark

In [9]:
def trata_tweets(df):
    words = df.select(f.explode(f.split(f.lower('_c0'), " ")).alias('word'))\
            .withColumn('word', f.regexp_replace('word', r'http\S+', ''))\
            .withColumn('word', f.regexp_replace('word', r'@\w+', ''))\
            .withColumn('word', f.regexp_replace('word', 'rt', ''))\
            .na.replace('', None)\
            .na.drop()
    return words             

In [ ]:
stops = stopwords.words('portuguese')
stops.append('futebol americano')
plt.figure(figsize=(20,10))

while True:
    try:
        words = spark.read.csv('./csv', encoding='utf-8')
        words =.trata_tweets(words)
        rows = words.collect()
        all_words = ''
        for row in rows:
            all_words = all_words + ' ' + row['word']

        wordcloud = WordCloud(stopwords=stops,
                            background_color='black',
                            width=1920,
                            height=1080,
                            max_words=100).generate(all_words)
                              
        plt.cla()
        plt.axis('off')
        plt.imshow(wordcloud)
        display.display(plt.gcf())
        display.clear_output(wait=True)
        time.sleep(5)
    except KeyboardInterrupt:
        break                         